# Классические методы



Вы каждый день используете функцию автокоррекции на своем мобильном телефоне и компьютере. В этом задании Вы реализуете аналогичную модель :)

<a name='0-1'></a>
### Установка библиотек и импорты

Установим сразу все необходимые библиотеки и скачаем нужные данные

In [ ]:
! pip install razdel tqdm datasets timeout-decorator
! pip install textdistance
! pip install pyaspeller

! git clone https://github.com/ai-forever/sage.git

! pip install tqdm

%cd sage
! pip install .
! pip install -e .[errant]

In [ ]:
%cd /content
!sudo apt-get install -y swig3.0
!pip install jamspell

!wget -nc https://github.com/bakwc/JamSpell-models/raw/master/ru.tar.gz
!tar -xf ru.tar.gz

In [ ]:
%load_ext autoreload
%autoreload 2

In [8]:
from collections import Counter
import json
import numpy as np
import os
import pandas as pd
from pprint import pprint
import re
from razdel import sentenize, tokenize as razdel_tokenize
import requests
from sklearn.metrics import classification_report, accuracy_score
from string import punctuation
import time
from tqdm import tqdm
import textdistance

<a name='0-1'></a>
### Метрика близости слов aka Edit Distance

Расстояние Левенштейна, или редакционное расстояние (англ.: edit distance), — метрика cходства между двумя строковыми последовательностями. Чем больше расстояние, тем более различны строки. Для двух одинаковых последовательностей расстояние равно нулю.

* Два слова находятся на расстоянии n друг от друга, если нужно совершить n изменений для превращения одного слова в другое.

Изменение может происходить следующим образом:

- Удаление буквы: ‘hat’ => ‘at, ha, ht’
- Замена буквы: ‘jat’ => ‘hat, rat, cat, mat, ...’
- Поменять две соседние буквы:  ‘eta’ => ‘eat, tea,...’
- Вставка буквы: ‘te’ => ‘the, ten, ate, ...’

Мы будем использовать данное расстояние, чтобы исправлять опечатки. Вместо того, чтобы генерировать все варианты исправления слова, можно искать похожие слова в словаре. Для этого нам нужно задать метрику похожести -- в данном ноутбуке это будет расстояние Левенштейна.

Чтобы реализовать автокорректор, нужно для каждого слова $w$ **уметь считать его наиболее вероятную коррекцию $c$**:

$$\underset{c \in W}{\mathrm{argmax}} \; P(c|w) = \underset{c \in W}{\mathrm{argmax}} \; P(w|c)\times  \frac{P(c)}{P(w)}  = \\ = \underset{c \in W}{\mathrm{argmax}} \; P(w|c)\times {P(c)} \tag{1}, $$


В уравнении (1) записана формула Байеса, где:

- $W$ -- все возможные корректные слова

- $P(c|w)$ -- вероятность того, что автор имел в виду слово $c$ при условии того, что напечатал $w$.

- $P(w|c)$ -- вероятность появления слова $w$ при условии того, что автор имел в виду $c$.

- $P(c)$ -- вероятность того, что слово $c$ появляется в тексте как корректное слово.

<a name='1'></a>
### Часть 0: Data Preprocessing

Как и в любой другой задаче машинного обучения, первое, что вам нужно сделать, - это обработать ваш набор данных. Мы будем строить автокорректор на основе данных с соревнования Dialog Evaluation 2016, в котором надо было исправить опечатки.


In [9]:
from datasets import load_dataset

dataset = load_dataset("ai-forever/spellcheck_benchmark", 'RUSpellRU', split='train')
dataset = dataset.to_pandas()[['source', 'correction']]

dataset

/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_token.py:89: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


README.md:   0%|          | 0.00/13.3k [00:00<?, ?B/s]

spellcheck_benchmark.py:   0%|          | 0.00/9.07k [00:00<?, ?B/s]

The repository for ai-forever/spellcheck_benchmark contains custom code which must be executed to correctly load the dataset. You can inspect the repository content at https://hf.co/datasets/ai-forever/spellcheck_benchmark.
You can avoid this prompt in future by passing the argument `trust_remote_code=True`.

Do you wish to run the custom code? [y/N] y


data/RUSpellRU/test.json:   0%|          | 0.00/1.95M [00:00<?, ?B/s]

data/RUSpellRU/train.json:   0%|          | 0.00/1.69M [00:00<?, ?B/s]

Generating train split: 0 examples [00:00, ? examples/s]

Generating test split: 0 examples [00:00, ? examples/s]

,source,correction
0,очень классная тетка ктобы что не говорил.,очень классная тетка кто бы что ни говорил
1,Может выгоднее втулку продать и купить колесо ...,Может выгоднее втулку продать и купить колесо ...
2,Довольно большая часть пришедших сходила с дор...,Довольно большая часть пришедших сходила с дор...
3,"Симпатичнейшое шпионское устройство, такой себ...",Симпатичнейшее шпионское устройство такой себе...
4,Опофеозом дня для меня сегодня стала фраза усл...,Апофеозом дня для меня сегодня стала фраза усл...
...,...,...
1995,Какой-то период времени мы вобще не общались...,Какой-то период времени мы вообще не общались
1996,Каковы ваши любимые и наименее любимые слова?,Каковы ваши любимые и наименее любимые слова
1997,"Сегодня яичницей никто не завтракал ( как, впр...",Сегодня яичницей никто не завтракал как впроче...
1998,Особое место занимает чудотворная икона « Лобз...,Особое место занимает чудотворная икона Лобзан...


Для обучения будем использовать только исправленные предложения (правый столбец).

In [30]:
texts = dataset['correction'].values.tolist()

punctuation = '.!?'
TEXT = '\n'.join([text if text[-1] in punctuation else text + '.' for text in texts])
print(TEXT[:402])

очень классная тетка кто бы что ни говорил.
Может выгоднее втулку продать и купить колесо в сборе.
Довольно большая часть пришедших сходила с дорожек и усаживалась на траву.
Симпатичнейшее шпионское устройство такой себе гламурный фотоаппарат девушки Бонда миниатюрная модель камеры Superheadz Clap Camera.
Апофеозом дня для меня сегодня стала фраза услышанная в новостях.
Ну не было поста так не было.


Приведем все к нижнему регистру, и получим список всех слов. В дальнейшем мы будем называть список всех слов из данного файла (= корпуса) вокабуляром (vocabulary).

In [31]:
def process_data(text):
    """
    Input:
        text: a string containing text.
    Output:
        words: a list containing all the words in the corpus (text you read) in lower case.
    """
    words = re.findall(r'\w+', text.lower())
    return words

In [32]:
word_l = process_data(TEXT)
vocab = set(word_l)  # this will be your new vocabulary

assert(len(vocab) == 9428)
print(f"The first ten words in the text are: \n{word_l[0:10]}")
print(f"There are {len(vocab)} unique words in the vocabulary.")

The first ten words in the text are: 
['очень', 'классная', 'тетка', 'кто', 'бы', 'что', 'ни', 'говорил', 'может', 'выгоднее']
There are 9428 unique words in the vocabulary.


Чтобы посчитать, сколько каждое слово встречается в тексте, построим словарь

In [33]:
def get_count(words):
    '''
    Input:
        word_l: a set of words representing the corpus.
    Output:
        word_count_dict: The wordcount dictionary where key is the word and value is its frequency.
    '''
    return dict(Counter(words))

WORDS = get_count(word_l)
print(f"There are {len(WORDS)} key values pairs")
print(f"The count for the word 'очень' is {WORDS.get('очень',0)}")

There are 9428 key values pairs
The count for the word 'очень' is 113


### Часть 1. Наивный подход

Вспомним задачу: для слова $w$ мы хотим найти его наиболее вероятную коррекцию.

$$\underset{c \in W}{\mathrm{argmax}} \; P(c|w) = \underset{c \in W}{\mathrm{argmax}} \; P(w|c)\times {P(c)} \tag{1}, $$


Можно решить задачу наивно: всегда будем брать более близкое слово (рассматриваем расстояния не более двух). Если таких слов несколько, берем слово с максимальной вероятностью. Если не нашлось ничего, то оставляем слово как есть.

Таким образом, идея алгоритма простая - для каждого неправильного слова нужно сгенерировать варианты исправлений и выбрать из них тот, что есть в словаре, а если таких несколько, то выбрать наиболее близкое (по введенной нами метрике) и наиболее вероятное.

In [34]:
def P(word, N=sum(WORDS.values())):
    "Probability of `word`."
    return WORDS.get(word, 0) / N

def known(words):
    "Вернуть подмножество слов, которое есть в нашем словаре."
    return set(w for w in words if w in WORDS)

def edits0(word):
    "Вернуть все строки, которые находятся на edit_distance == 0 от word (т.е., просто само слово)."
    return {word}

In [35]:
def edits1(word):
    """
    Возвращает множество слов, находящихся на расстоянии edit_distance == 1 от word.

    Параметры:
    word (str): Исходное слово.

    Возвращает:
    set: Множество слов на расстоянии редактирования 1.
    """
    # Плейсхолдер: возвращаем пустое множество
    return set()

def edits2(word):
    """
    Возвращает множество слов, находящихся на расстоянии edit_distance == 2 от word.

    Параметры:
    word (str): Исходное слово.

    Возвращает:
    set: Множество слов на расстоянии редактирования 2.
    """
    # Плейсхолдер: возвращаем пустое множество
    return set()

def candidates(word):
    "Generate possible spelling corrections for word."
    return (known(edits0(word)) or
            known(edits1(word)) or
            known(edits2(word)) or
            [word])

def correction(word):
    "Most probable spelling correction for word."
    return max(candidates(word), key=P)

Как генерировать исправления?

**Задание:** реализуйте функции edits1 и edits2, которые возвращают исправления, находящиеся на расстоянии 1 и 2, соответственно, от исходного слова

Функция edits1(word) должна возвращать множество слов, находящихся на расстоянии edit_distance == 1. Слова, находящиеся на расстоянии 1 от исходного -- это множество всех возможных слов, которые можно получить, совершив одну из следующих операций:

1.	Удаление одной буквы: возвращает все возможные строки, где одна буква удалена.
2.	Перестановка соседних букв: возвращает все возможные строки, где две соседние буквы поменяны местами.
3.	Замена одной буквы: возвращает все возможные строки, где одна буква заменена на другую.
4.	Вставка дополнительной буквы: возвращает все возможные строки, где добавлена одна буква в любое место.

Например, для слова "wird" результат будет включать такие слова, как:

•	"wrd" (удаление буквы i),

•	"iwrd" (перестановка соседних w и i),

•	"word" (замена буквы i на o),

•	"weird" (вставка буквы e),

Чтобы получить только существующие слова, можно воспользоваться функцией known.

*Как можно реализовать такие функции?*

 Например можно *разбить* исходное слово на пару всеми возможными способами (каждое *разбиение* даст нам пару "слов"), `(a, b)`, первая часть - до места разбиения, а вторая - после, и в каждом месте разбиения можно: удалить, поменять местами, заменить или вставить букву:

<table>
  <tr><td> пары: <td><tt> Ø+wird <td><tt> w+ird <td><tt> wi+rd <td><tt>wir+d<td><tt>wird+Ø<td><i>Notes:</i><tt> (<i>a</i>, <i>b</i>)</tt> пара</i>
  <tr><td> удаления: <td><tt>Ø+ird<td><tt> w+rd<td><tt> wi+d<td><tt> wir+Ø<td><td><i>Удаление первой буквы в b</i>
  <tr><td> перемена мест: <td><tt>Ø+iwrd<td><tt> w+rid<td><tt> wi+dr</tt><td><td><td><i>Перемена мест двух первых букв b
  <tr><td> замена: <td><tt>Ø+?ird<td><tt> w+?rd<td><tt> wi+?d<td><tt> wir+?</tt><td><td><i>замена буквы в начале b
  <tr><td> вставка: <td><tt>Ø+?+wird<td><tt> w+?+ird<td><tt> wi+?+rd<td><tt> wir+?+d<td><tt> wird+?+Ø</tt><td><i>Вставка буквы между a и b
</table>

Реализуйте функцию delete_letter(), которая, получив слово, возвращает список строк с удаленным одним символом

Например, для слова **nice**, функция должна вернуть: {'ice', 'nce', 'nic', 'nie'}.

**Шаг 1:** Сделать список 'splits': то, как можно разделить слово на "лево" и "право". Например,   
nice можно разделить на : `[('', 'nice'), ('n', 'ice'), ('ni', 'ce'), ('nic', 'e'), ('nice', '')]`

**Шаг 2:** Сгенерировать список всех слов, которые могут быть получены после удаления одной буквы

In [37]:
def splits(word):
    "Возвращает список всех возможных разбиений слова на пару (a, b)."
    return [(word[:i], word[i:])
            for i in range(len(word)+1)]

split = splits('nice')
split

[('', 'nice'), ('n', 'ice'), ('ni', 'ce'), ('nic', 'e'), ('nice', '')]

In [38]:
def delete_letter(word, split, verbose=False):
  delete = [L + R[1:] for L, R in split if R]
  if verbose: print(f"input word {word}, \nsplit = {split}, \ndelete = {delete}")
  return delete

In [39]:
delete_word = delete_letter(word="nice", split=splits('nice'),
                        verbose=True)

input word nice, 
split = [('', 'nice'), ('n', 'ice'), ('ni', 'ce'), ('nic', 'e'), ('nice', '')], 
delete = ['ice', 'nce', 'nie', 'nic']


По аналогии реализуйте функции remove_letter, transpose_letters, replace_letter, insert_letter

In [ ]:
def transpose_letters(word, split, verbose=False):
  transpose = #YOUR CODE IS HERE
  if verbose: print(f"input word {word}, \nsplit = {split}, \ntranspose = {transpose}")
  return transpose

In [ ]:
def replace_letter(word, split, verbose=False, alphabet='йцукенгшщзхъфывапролджэячсмитьбюё'):
  remove = #YOUR CODE IS HERE
  if verbose: print(f"input word {word}, \nsplit = {split}, \nremove = {remove}")
  return remove

In [ ]:
def insert_letter(word, split, alphabet='йцукенгшщзхъфывапролджэячсмитьбюё', verbose=False):
  insert = #YOUR CODE IS HERE
  if verbose: print(f"input word {word}, \nsplit = {split}, \ninsert = {insert}")
  return insert

In [40]:
# Проверки для слова "nice"
assert delete_letter("nice", split=splits('nice')) == ['ice', 'nce', 'nie', 'nic'], "Ошибка в функции delete_letter"
assert transpose_letters("nice", split=splits('nice')) == ['ince', 'ncie', 'niec'], "Ошибка в функции transpose_letters"

expected_replacements = {
    'aice', 'bice', 'cice',  # Замены для 'n'
    'nace', 'nbce', 'ncce',  # Замены для 'i'
    'niae', 'nibe', 'nice',  # Замены для 'c', включая исходное слово, так как 'c' есть в алфавите
    'nica', 'nicb', 'nicc'   # Замены для 'e'
}
assert set(replace_letter("nice", split=splits('nice'), alphabet='abc')) == expected_replacements, "Ошибка в функции replace_letter"

expected_insertions = {
    'anice', 'bnice', 'cnice',  # Вставки перед 'n'
    'naice', 'nbice', 'ncice',  # Вставки между 'n' и 'i'
    'niace', 'nibce', 'nicce',  # Вставки между 'i' и 'c'
    'nicae', 'nicbe', 'nicce',  # Вставки между 'c' и 'e'
    'nicea', 'niceb', 'nicec'   # Вставки после 'e'
}
assert set(insert_letter("nice", split=splits('nice'), alphabet='abc')) == expected_insertions, "Ошибка в функции insert_letter"

print("Все проверки пройдены успешно.")

Все проверки пройдены успешно.


In [41]:
def edits1(word, alphabet='йцукенгшщзхъфывапролджэячсмитьбюё'):
    """
    Input:
        word: the string/word for which we will generate all possible wordsthat are one edit away.
    Output:
        edit_one_set: a set of words with one possible edit. Please return a set. and not a list.
    """

    split = splits(word)
    replaces = replace_letter(word, split, alphabet=alphabet)
    inserts = insert_letter(word, split, alphabet=alphabet)
    deletes = delete_letter(word, split)
    transposes = transpose_letters(word, split)

    return set(deletes + transposes + replaces + inserts)

In [42]:
expected_results = {
    # Удаления
    'ice', 'nce', 'nie', 'nic',
    # Транспозиции
    'ince', 'ncie', 'niec',
    # Замены (с упрощенным алфавитом 'abc')
    'aice', 'bice', 'cice', 'nace', 'nbce', 'ncce', 'niae', 'nibe', 'nice', 'nica', 'nicb', 'nicc',
    # Вставки (с упрощенным алфавитом 'abc')
    'anice', 'bnice', 'cnice', 'naice', 'nbice', 'ncice', 'niace', 'nibce', 'nicce', 'nicae', 'nicbe', 'nicce', 'nicea', 'niceb', 'nicec'
}
assert edits1("nice", alphabet='abc') == expected_results, "Ошибка в функции edit_one_letter"

Теперь реализуем функцию, которая получает все слова на расстоянии 2.

**Hint**. Возьмите слова, полученные с помощью edit_one_letter (они находятся на расстоянии 1 от исходного слова), и примените к ним    функцию edit_one_letter. Полученные слова будут находиться на расстоянии 2 от исходного.


In [43]:
def edits2(word):
    "Вернуть все строки, которые находятся на edit_distance == 2 от word."
    result_set = set()

    for e1 in edits1(word): #все слова на расстоянии 1
        for e2 in edits1(e1): #все слова на расстоянии 2
            result_set.add(e2)

    return result_set

In [44]:
import re

def correct_match(match):
    """Исправить слово word в match-группе без изменения регистра."""
    word = match.group()
    return correction(word.lower())

def correct_text(text):
    """Исправить все слова с опечатками в тексте на русском языке."""
    return re.sub('[а-яёА-ЯЁ]+', correct_match, text)

text = "Превет мир! Как дила?"
corrected_text = correct_text(text)
print(corrected_text)

привет мир! как дела?


Идеи для улучшения:

1) Использовать н-граммы, чтобы учитывать контекст. [Пример](https://nbviewer.org/url/norvig.com/ipython/How%20to%20Do%20Things%20with%20Words.ipynb)

2) Использовать предобученные векторы, чтобы учитывать контекст, например, word2vec или fasttext [Пример](https://blog.reachsumit.com/posts/2020/07/spell-checker-fasttext/#method-1-using-pre-trained-word-vectors)

# Машинное обучение

**Jamspell**

Воспользуемся моделью, которую предложили авторы метода. Они отмечают, что лучше обучить модель на своих данных, но для обзора можем воспользоваться их весами.

In [ ]:
import jamspell

corrector = jamspell.TSpellCorrector()
corrector.LoadLangModel('ru_small.bin')

def correct_jamspell(text):
    return corrector.FixFragment(text)

In [45]:
correct_jamspell('Превет мир! Как дила?')

'Привет мир! Как дила?'

In [ ]:
samples = [
    "прийдя в МГТУ я был удивлен никого необноружив там…",
    "Нащщот Чавеса разве что не соглашусь.",
    "Мошный лазер - в нерабочем состоянии - 350 кредиток.",
    "Ощушаю себя с ними монголойдом, я никогда так много не молчала как молчю тут, и не потому, что языковый баръер или еще что-то, просто коментариев нет"
]

for sample in samples:
  corrected_j = correct_jamspell(sample)
  corrected_n = correct_text(sample)
  print(f"Строка: {sample}")
  print(f"Исправление jamspell: {corrected_j}")
  print(f"Исправление наивным: {corrected_n}")

Строка: прийдя в МГТУ я был удивлен никого необноружив там…
Исправление jamspell: придя в МГТУ я был удивлен никого необноружив там…
Исправление наивным: придя в мгту я был удивлен никого необноружив там…
Строка: Нащщот Чавеса разве что не соглашусь.
Исправление jamspell: Наото Чавеса разве что не соглашусь.
Исправление наивным: нащщот чавеса разве что не соглашусь.
Строка: Мошный лазер - в нерабочем состоянии - 350 кредиток.
Исправление jamspell: Мощный лазер - в нерабочем состоянии - 350 кредиток.
Исправление наивным: мощный лазер - в нерабочем состоянии - 350 кредиток.
Строка: Ощушаю себя с ними монголойдом, я никогда так много не молчала как молчю тут, и не потому, что языковый баръер или еще что-то, просто коментариев нет
Исправление jamspell: Ощущаю себя с ними монголойдом, я никогда так много не молчала как молча тут, и не потому, что языковый барьер или еще что-то, просто коментариев нет
Исправление наивным: ощущаю себя с ними монголоидом, я никогда так много не молчала как мол

На первый взгляд, наивная реализация и jamspell одинаково плохи (для jamspell, скорее всего, это связано с малой обучающей выборкой), но, возможно, jamspell покажет себя чуть лучше в обработке контекста.

**SAGE**

In [ ]:
import os
import torch
from sage.spelling_correction import T5ModelForSpellingCorruption, RuM2M100ModelForSpellingCorrection, AvailableCorrectors

print(*["{}: {}".format(item.name, item.value) for item in AvailableCorrectors], sep="\n")

sage_fredt5_large: ai-forever/sage-fredt5-large
sage_fredt5_distilled_95m: ai-forever/sage-fredt5-distilled-95m
sage_m2m100_1B: ai-forever/sage-m2m100-1.2B
sage_mt5_large: ai-forever/sage-mt5-large
m2m100_1B: ai-forever/RuM2M100-1.2B
m2m100_418M: ai-forever/RuM2M100-418M
fred_large: ai-forever/FRED-T5-large-spell
ent5_large: ai-forever/T5-large-spell


In [ ]:
sage_m2m100_corrector = RuM2M100ModelForSpellingCorrection.from_pretrained(AvailableCorrectors.sage_m2m100_1B.value)
sage_fredt5_large_corrector = T5ModelForSpellingCorruption.from_pretrained(AvailableCorrectors.sage_mt5_large.value)
sage_fredt5_95m_corrector = T5ModelForSpellingCorruption.from_pretrained(AvailableCorrectors.sage_fredt5_distilled_95m.value)

/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_token.py:89: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(
You set `add_prefix_space`. The tokenizer needs to be converted from the slow tokenizers
/usr/local/lib/python3.10/dist-packages/transformers/convert_slow_tokenizer.py:551: UserWarning: The sentencepiece tokenizer that you are converting to a fast tokenizer uses the byte fallback option which is not implemented in the fast tokenizers. In practice this means that the fast version of the tokenizer can produce unknown tokens whereas the sentencepiece version would have converted 

In [ ]:
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
sage_m2m100_corrector.model.to(device);
sage_fredt5_large_corrector.model.to(device);
sage_fredt5_95m_corrector.model.to(device);

In [ ]:
correctors = {sage_m2m100_corrector: 'sage_m2m100_corrector',
              sage_fredt5_large_corrector: 'sage_fredt5_large_corrector',
              sage_fredt5_95m_corrector: 'sage_fredt5_95m_corrector',
              }

In [ ]:
samples = [
    "прийдя в МГТУ я был удивлен никого необноружив там…",
    "Нащщот Чавеса разве что не соглашусь.",
    "Мошный лазер - в нерабочем состоянии - 350 кредиток.",
    "Ощушаю себя с ними монголойдом, я никогда так много не молчала как молчю тут, и не потому, что языковый баръер или еще что-то, просто коментариев нет"
]

In [ ]:
import time
def generate_fixed_from_samples(model, tokenizer, samples, device='cpu'):
    model = model.to(device)
    t_start = time.perf_counter()

    tokens = tokenizer(samples, padding=True, return_tensors='pt')
    output = model.generate(tokens['input_ids'].to(device), do_sample=True, top_k=50, top_p=0.95, num_return_sequences=1)
    results = tokenizer.batch_decode(output.cpu(), skip_special_tokens=True)

    all_time = time.perf_counter() - t_start
    return dict(zip(samples, results)), all_time

In [ ]:
results = dict()
for corrector, corrector_name in correctors.items():
    tokenizer = corrector.tokenizer
    model = corrector.model

    fixed_samples, time_for_gen_gpu = generate_fixed_from_samples(model, tokenizer, samples, device=device)

    print(f"Time for gpu inference, model {corrector_name}", round(time_for_gen_gpu, 2))
    results[corrector_name] = fixed_samples

Time for gpu inference, model sage_m2m100_corrector 1.52
Time for gpu inference, model sage_fredt5_large_corrector 2.22
Time for gpu inference, model sage_fredt5_95m_corrector 0.63


In [ ]:
unique_originals = set()
for corrections in results.values():
    unique_originals.update(corrections.keys())

for original in unique_originals:
    print(f"Строка: '{original}'")
    for model_name, corrections in results.items():
        corrected = corrections.get(original)
        print(f"{model_name}: '{corrected}'")
    print()

Строка: 'прийдя в МГТУ я был удивлен никого необноружив там…'
sage_m2m100_corrector: 'придя в МГТУ я был удивлен никого не обнаружив там'
sage_fredt5_large_corrector: 'Придя в МГТУ, я был удивлен, никого не обнаружив там...'
sage_fredt5_95m_corrector: 'Придя в МГТУ, я был удивлён, никого не обнаружив там.'

Строка: 'Мошный лазер - в нерабочем состоянии - 350 кредиток.'
sage_m2m100_corrector: 'Мощный лазер в нерабочем состоянии 350 кредиток'
sage_fredt5_large_corrector: 'Мощный лазер в нерабочем состоянии - 360 кредиток.'
sage_fredt5_95m_corrector: 'Мощный лазер в нерабочем состоянии - 350 кредиток.'

Строка: 'Нащщот Чавеса разве что не соглашусь.'
sage_m2m100_corrector: 'Насчёт Чавеса разве что не соглашусь'
sage_fredt5_large_corrector: 'Насчет Чавеса разве что не соглашусь.'
sage_fredt5_95m_corrector: 'Насчёт Чавеса разве что не соглашусь.'

Строка: 'Ощушаю себя с ними монголойдом, я никогда так много не молчала как молчю тут, и не потому, что языковый баръер или еще что-то, просто ко

# Сравнение моделей

Посчитаем метрики f1, precision, recall для всех моделей, которые мы сегодня рассмотрели, на тестовой части датасета RuSpellEval. Функция evaluation принимает на вход список строк, которые надо исправить (sources), исправлений моделью (corrections), и верных исправлений  (answers), и возвращает метрики:

Recall: какую долю ошибок модель нашла и исправила = $\frac{\text{правильные исправления}}{\text{всего ошибок в тексте}}$

Precision: какая доля исправлений, предложенных моделью, была правильной = $\frac{\text{правильные исправления}}{\text{всего предложенных исправлений}}$


In [46]:
!wget https://raw.githubusercontent.com/ai-forever/sage/main/sage/evaluation/ruspelleval.py

--2024-10-23 22:08:31--  https://raw.githubusercontent.com/ai-forever/sage/main/sage/evaluation/ruspelleval.py
Resolving raw.githubusercontent.com (raw.githubusercontent.com)... 185.199.108.133, 185.199.109.133, 185.199.110.133, ...
Connecting to raw.githubusercontent.com (raw.githubusercontent.com)|185.199.108.133|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 19418 (19K) [text/plain]
Saving to: ‘ruspelleval.py’

ruspelleval.py      100%[===================>]  18.96K  --.-KB/s    in 0.001s  

2024-10-23 22:08:32 (31.7 MB/s) - ‘ruspelleval.py’ saved [19418/19418]



In [47]:
from ruspelleval import evaluation
evaluation

<function ruspelleval.evaluation(sources: List[str], corrections: List[str], answers: List[str], to_output_differences: bool = False, path_to_diff: str = 'diff.txt') -> Dict[str, float]>

Тут нам уже понадобятся оба столбца. Будем просить модель исправить данные из столбца source. Идеальная модель должна получить тексты, совпадающие с данными из столбца correction.

In [48]:
dataset = load_dataset("ai-forever/spellcheck_benchmark", 'RUSpellRU', split='test')
dataset = dataset.to_pandas()[['source', 'correction']]
sources = dataset['source'].values.tolist()
answers = dataset['correction'].values.tolist()

In [ ]:
dataset.head()

,source,correction
0,﻿есть у вас оформленый и подписаный мною заказ,﻿есть у вас оформленный и подписанный мною заказ
1,вот в инете откапал такую интеерсную статейку ...,вот в инете откопал такую интересную статейку ...
2,я на всю жизнь запомню свое первое купание в з...,я на всю жизнь запомню свое первое купание в з...
3,думаем что не ошибемся если скажем что выставк...,думаем что не ошибемся если скажем что выставк...
4,судьба человека может складываться очень разно...,судьба человека может складываться очень разно...


In [49]:
from tqdm import tqdm

jamspell_cor = []
naive_cor = []

for text in tqdm(sources):
  jamspell_c = correct_jamspell(text)
  naive = correct_text(text)
  jamspell_cor.append(jamspell_c)
  naive_cor.append(naive)

100%|██████████| 2008/2008 [36:24<00:00,  1.09s/it]


In [50]:
jamespell_metrics = evaluation(sources, jamspell_cor, answers)
naive_metrics = evaluation(sources, naive_cor, answers)

Calculating words metric:   0%|          | 0/2008 [00:00<?, ?it/s]

Calculating words metric:   0%|          | 0/2008 [00:00<?, ?it/s]

In [51]:
jamespell_metrics

{'Precision': 32.81, 'Recall': 41.9, 'F1': 36.8}

In [52]:
naive_metrics

{'Precision': 29.99, 'Recall': 8.7, 'F1': 13.48}

Видим, что у jamespell получилось найти и исправить значительно больше ошибок (так как recall выше), сохраняя точност предложенных исправлений.

Посмотрим на самую маленькую модель из sage

In [ ]:
corrector = sage_fredt5_95m_corrector
corrector_name = 'sage_fredt5_95m_corrector'
corrector.model.to("cuda:0")

metrics = corrector.evaluate("RUSpellRU", batch_size=16, metrics=["ruspelleval"])
print(f"Metrics for {corrector_name}:")
print(metrics)

  0%|          | 0/126 [00:00<?, ?it/s]

Calculating words metric:   0%|          | 0/2008 [00:00<?, ?it/s]

Metrics for sage_fredt5_95m_corrector:
{'Precision': 83.48, 'Recall': 74.75, 'F1': 78.87}


Получается значительно лучше =)

Сравним метрики с [Яндекс Спеллером](https://yandex.ru/dev/speller/), который тоже использует CatBoost для исправления ошибок.

In [53]:
!pip install pyaspeller

In [54]:
from pyaspeller import YandexSpeller

yaspeller_cor = []

speller = YandexSpeller()

for text in tqdm(sources):
  yaspeller = speller.spelled(text)
  yaspeller_cor.append(yaspeller)

100%|██████████| 2008/2008 [09:28<00:00,  3.53it/s]


In [55]:
yaspeller_metrics = evaluation(sources, yaspeller_cor, answers)

Calculating words metric:   0%|          | 0/2008 [00:00<?, ?it/s]

In [56]:
yaspeller_metrics

{'Precision': 60.7, 'Recall': 83.04, 'F1': 70.13}